# Task Decomposition with Two Agents

In this exploration we create an Assistant agent with the ability to use a third agent to help with decomposing (breaking up) a task into individual steps and then executing each of those steps with minimal human input. 

We are again picking the task of analysing the iris dataset and creating a predictive model. Instead of jumping into creating the model directly we will ask the agents to do exploratory data analysis then train the model and show the outputs. This would help us, as the supervisor to make sense of the work. 

The whole task below took about **10min to execute** including the time for human input.

To see the results of the output in a nicely formatted blog post **generated by the AI*** go to [the next page.](https://ai.aneeshsathe.com/Two%20Agent%20Task%20Decomposition/AI%20Auto%20Blog%20Post%20Logistic%20Regression.html)

In [2]:
import os

from typing_extensions import Annotated

from autogen import (
    AssistantAgent,
    UserProxyAgent,
    config_list_from_json,
    register_function,
)

from autogen.cache import Cache
from autogen.coding import DockerCommandLineCodeExecutor, LocalCommandLineCodeExecutor

from pathlib import Path

from autogen import ConversableAgent


work_dir = "two_agent_task_decomp"

In [3]:
# model = "llama3:latest"

# config_list = [
#     {
#         "model": model,
#         "base_url": "http://localhost:11434/v1",
#         "api_key": "ollama", # required, but not used,
#     }
# ]
# llm_config = {"config_list": config_list, "cache_seed": None}

In [4]:
config_list_path = "json_configs/OAI_CONFIG_LIST.json"
config_list = config_list_from_json(env_or_file=config_list_path)

In [5]:
task = (
    f"Do exploratory data analysis of the iris dataset, save any plots you produce as jpg files. Then create a 3 class logistic regression model. Show the performance of the model as a plot showing the decision boundaries in individual background colors and datapoints colored by label. Finally write a short blog post about the iris dataset and the process used for data exploration and creating the predictive model. Use the task_planner function to break down task into substeps. Provide bash scripts to install libraries before code execution."
)
# print(task)

## Approach 1. Two-agent chat with function call for task decomposition

In this approach, we use a planner agent for coming up a plan.
The planner agent is wrapped inside a function to be used as a tool.

In [6]:
# Create planner agent.
planner = AssistantAgent(
    name="planner",
    llm_config={
        "config_list": config_list,
        "cache_seed": None,  # Disable legacy cache.
    },
    system_message="You are a helpful AI assistant. You suggest a feasible plan "
    "for finishing a complex task by decomposing it into 3-5 sub-tasks. "
    "If the plan is not good, suggest a better plan. "
    "If the execution is wrong, analyze the error and suggest a fix.",
)

# Create a planner user agent used to interact with the planner.
planner_user = UserProxyAgent(
    name="planner_user",
    human_input_mode="NEVER",
    code_execution_config=False,
)

# The function for asking the planner.


def task_planner(question: Annotated[str, "Question to ask the planner."]) -> str:
    with Cache.disk(cache_seed=4) as cache:
        planner_user.initiate_chat(planner, message=question, max_turns=1, cache=cache)
    # return the last message received from the planner
    return planner_user.last_message()["content"]

In [7]:
# task_planner("how do i do data exploration for iris dataset?")

Next, we create an assistant agent to execute the plan, using the planner agent as a tool.

In [8]:
# Create assistant agent.
assistant = AssistantAgent(
    name="assistant",
    system_message="You are a helpful AI assistant. "
    "You can use the task planner to decompose a complex task into sub-tasks. "
    "Make sure your follow through the sub-tasks. "
    "When needed, write Python code in markdown blocks, and I will execute them."
    "Give the user a final solution at the end. "
    "Return TERMINATE only if the sub-tasks are completed.",
    llm_config={
        "config_list": config_list,
        "cache_seed": None,  # Disable legacy cache.
    },
)





# Setting up code executor.
os.makedirs(work_dir, exist_ok=True)
# Use DockerCommandLineCodeExecutor to run code in a docker container.
code_executor = DockerCommandLineCodeExecutor(work_dir=work_dir)
# code_executor = LocalCommandLineCodeExecutor(work_dir=work_dir)

# Create user proxy agent used to interact with the assistant.
user_proxy = UserProxyAgent(
    name="user_proxy",
    system_message="Give approvals and execute code from the assistant.",
    human_input_mode="ALWAYS",
    is_termination_msg=lambda x: "content" in x
    and x["content"] is not None
    and x["content"].rstrip().endswith("TERMINATE"),
    code_execution_config={"executor": code_executor},
)

# Register the function to the agent pair.
register_function(
    task_planner,
    caller=assistant,
    executor=user_proxy,
    name="task_planner",
    description="A task planner than can help you with breaking down a complex task into sub-tasks.",
)

Start the task

In [9]:
# user_proxy.initiate_chat(
#     assistant,
#     message=task,
#     # cache=cache,
# )

In [10]:
# Use Cache.disk to cache LLM responses. Change cache_seed for different responses.
with Cache.disk(cache_seed=1) as cache:
    # the assistant receives a message from the user, which contains the task description
    user_proxy.initiate_chat(
        assistant,
        message=task,
        cache=cache,
    )

user_proxy (to assistant):

Do exploratory data analysis of the iris dataset, save any plots you produce as jpg files. Then create a 3 class logistic regression model. Show the performance of the model as a plot showing the decision boundaries in individual background colors and datapoints colored by label. Finally write a short blog post about the iris dataset and the process used for data exploration and creating the predictive model. Use the task_planner function to break down task into substeps. Provide bash scripts to install libraries before code execution.

--------------------------------------------------------------------------------
assistant (to user_proxy):

***** Suggested tool call (call_0IGb0jp8XonxUWn0B3ZoxUwB): task_planner *****
Arguments: 
{"question":"How do I perform exploratory data analysis on the iris dataset, save plots, create a logistic regression model, and write a blog post?"}
*****************************************************************************

-

See next page for the AI generated blog-post!